In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Model
from tensorflow.keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
df = pd.read_csv('blurb_top19.csv',delimiter=',',encoding='latin-1')
label_map = {}
index = 0
for label_item in df.label:
    try:
        label_map[label_item]
    except KeyError:
        label_map[label_item] = index
        index = index + 1
df.replace(label_map, inplace=True)

In [3]:
train_df, test_df = train_test_split(df, train_size=0.8, stratify=df['label'])
X_train = train_df['text']
X_test = test_df['text']
y_train = train_df['label']
y_test = test_df['label']

In [4]:
le = LabelEncoder()
y_train = le.fit_transform(y_train).reshape(-1,1)
y_test = le.fit_transform(y_test).reshape(-1,1)

In [5]:
max_words = 10000
max_len = 150
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X_train)
sequences = tok.texts_to_sequences(X_train)
sequences_matrix = sequence.pad_sequences(sequences,maxlen=max_len)

In [6]:
def RNN():
    inputs = Input(name='inputs',shape=[max_len])
    layer = Embedding(max_words,50,input_length=max_len)(inputs)
    layer = LSTM(64)(layer)
    layer = Dense(256,name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(18,name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs,outputs=layer)
    return model

In [7]:
model = RNN()
model.summary()
model.compile(loss='sparse_categorical_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inputs (InputLayer)          [(None, 150)]             0         
_________________________________________________________________
embedding (Embedding)        (None, 150, 50)           500000    
_________________________________________________________________
unified_lstm (UnifiedLSTM)   (None, 64)                29440     
_________________________________________________________________
FC1 (Dense)                  (None, 256)               16640     
_________________________________________________________________
activation (Activation)      (None, 256)               0         
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
out_layer (Dense)            (None, 18)                4626  

In [8]:
model.fit(sequences_matrix,y_train,batch_size=128,epochs=10,
          validation_split=0.2)

Train on 26173 samples, validate on 6544 samples
Epoch 1/10
26173/26173 [==============================] - 34s 1ms/sample - loss: 2.4782 - accuracy: 0.1416 - val_loss: 2.3335 - val_accuracy: 0.1754
Epoch 2/10
26173/26173 [==============================] - 28s 1ms/sample - loss: 2.1539 - accuracy: 0.1925 - val_loss: 2.0530 - val_accuracy: 0.2028
Epoch 3/10
26173/26173 [==============================] - 27s 1ms/sample - loss: 1.9378 - accuracy: 0.2781 - val_loss: 1.9478 - val_accuracy: 0.2937
Epoch 4/10
26173/26173 [==============================] - 27s 1ms/sample - loss: 1.7330 - accuracy: 0.3530 - val_loss: 1.7885 - val_accuracy: 0.3545
Epoch 5/10
26173/26173 [==============================] - 27s 1ms/sample - loss: 1.5187 - accuracy: 0.4500 - val_loss: 1.5825 - val_accuracy: 0.4542
Epoch 6/10
26173/26173 [==============================] - 27s 1ms/sample - loss: 1.3066 - accuracy: 0.5270 - val_loss: 1.5236 - val_accuracy: 0.4892
Epoch 7/10
26173/26173 [==============================] -